THIS NOTEBOOK USES GEMINI AS THE PREFERED LLM 

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")

# Now initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0.0
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
template_1 = """You are a video game expert. Recommend 5 {genre} games similar to {type}.

Requirements:
- Only list game titles (one per line)
- No descriptions, ratings, or extra text
- Focus on critically acclaimed games
- Include both classic and modern titles

Format:
1. Game Title
2. Game Title
..."""

template_2 = """Below is a list of video games:

{games}

For each game above, provide:
- A 1-2 sentence description
- Key gameplay features or what makes it unique

Format each entry as:
**Game Title**: Description here.

Important: Only describe the games listed. Do not recommend additional titles."""



In [4]:
from langchain.prompts import PromptTemplate

# Template for game recommendation
prompt_temp_genre_1 = PromptTemplate(
    input_variables=["genre"],
    template=template_1,
    partial_variables= {"type" : ""} # This is a optional variable 
)

# Template for description generation
prompt_temp_genre_2 = PromptTemplate(
    input_variables=["games"],
    template=template_2,
)


Simple Chain

In [5]:
# SIMPLE CHAIN

chain_1 = prompt_temp_genre_1 | llm | StrOutputParser()
chain_2 = prompt_temp_genre_2 | llm | StrOutputParser()

result = chain_1.invoke({"genre": "rpg", "type": "souls" })
print(result)

Bloodborne
Nioh 2
Dragon's Dogma: Dark Arisen
Sekiro: Shadows Die Twice
Lies of P


Simple Sequential chain

In [6]:
seq_chain = chain_1 | (lambda games: {"games": games}) | chain_2

result_seq = seq_chain.invoke({"genre": "rpg"})
result_seq_2 = seq_chain.invoke({"genre": "rpg", "type": "souls"})
print(result_seq)

Here are the descriptions and key features for each game:

**The Witcher 3: Wild Hunt**: An action role-playing game set in a vast, morally ambiguous fantasy world, where players control Geralt of Rivia, a monster hunter for hire. He embarks on an epic quest to find his adopted daughter Ciri while navigating political intrigue and supernatural threats.
Key gameplay features or what makes it unique:
*   Expansive open-world exploration with dynamic weather and day/night cycles.
*   Deep, branching narrative with player choices impacting the story and world.
*   Action combat system combining swordplay, magic (Signs), alchemy, and monster knowledge.

**Final Fantasy VII**: A seminal Japanese role-playing game that follows Cloud Strife, a mercenary with a mysterious past, as he joins an eco-terrorist group to fight against the corrupt Shinra Electric Power Company. Their struggle soon escalates into a battle to save the planet from a greater cosmic threat.
Key gameplay features or what ma

Sequential chain or combined chain ( output of 1st chain is input of the 2nd)

In [7]:
game_chain = prompt_temp_genre_1 | llm | StrOutputParser()
des_chain = prompt_temp_genre_2 | llm | StrOutputParser()

combined_chain = (
    game_chain  # Step 1

    | (lambda games: {  # Step 2 (depends on Step 1)
        "games": games,

        "descriptions": des_chain.invoke({"games": games})  # Step 3 (depends on Step 1)
    })
)

result = combined_chain.invoke({"genre": "rpg", "type": "souls"})

print("Games:\n", result["games"])
print("\nDescriptions:\n", result["descriptions"])

Games:
 Bloodborne
Nioh 2
Dragon's Dogma: Dark Arisen
Sekiro: Shadows Die Twice
Lies of P

Descriptions:
 **Bloodborne**: An action RPG set in the gothic, Victorian-era city of Yharnam, where players hunt monstrous beasts and unravel a cosmic horror narrative. It's known for its aggressive combat and oppressive atmosphere.
Key gameplay features: Fast-paced, offensive combat encouraging players to attack to regain health; trick weapons that transform between two distinct forms; a unique "Regain" system for health recovery; a strong emphasis on dodging and parrying with a firearm.

**Nioh 2**: An action RPG set in a fictionalized Sengoku-era Japan, where players control a half-human, half-Yokai protagonist fighting demons and historical figures. It features deep combat mechanics and extensive loot.
Key gameplay features: Stance-based combat (high, mid, low) that alters attack patterns and defense; the "Ki Pulse" mechanic to quickly recover stamina; Yokai abilities allowing players to tra